Jan 11, 2021

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pandas as pd
import numpy as np

from pyspark.sql.functions import when, udf, col, regexp_extract, regexp_replace,count,sum,avg,round
from pyspark.sql.types import DoubleType,IntegerType, StringType

# 스파크 통계
import pyspark.sql.functions as F

In [2]:
sc = SparkContext( 'local' ) 
sqlCtx = SQLContext( sc )

In [3]:
emp = [ ("Lisa", "Sales", 10000, 35),
("Evan", "Sales", 32000, 38),
 ("Fred", "Engineering", 21000, 28),
("Alex", "Sales", 30000, 33),
("Tom", "Engineering", 23000, 33),
 ("Jane", "Marketing", 29000, 28),
("Jeff", "Marketing", 35000, 38),
 ("Paul", "Engineering", 29000, 23),
("Chloe", "Engineering", 23000, 25)]

In [4]:
empDF = sqlCtx.createDataFrame( emp,('name' , 'dept' , 'salary' , 'age' ))
empDF.show()

+-----+-----------+------+---+
| name|       dept|salary|age|
+-----+-----------+------+---+
| Lisa|      Sales| 10000| 35|
| Evan|      Sales| 32000| 38|
| Fred|Engineering| 21000| 28|
| Alex|      Sales| 30000| 33|
|  Tom|Engineering| 23000| 33|
| Jane|  Marketing| 29000| 28|
| Jeff|  Marketing| 35000| 38|
| Paul|Engineering| 29000| 23|
|Chloe|Engineering| 23000| 25|
+-----+-----------+------+---+



In [5]:
empDF.createOrReplaceTempView('employees')

In [8]:
sql = 'select * from employees'
sqldf = sqlCtx.sql( sql )
sqldf.show()

+-----+-----------+------+---+
| name|       dept|salary|age|
+-----+-----------+------+---+
| Lisa|      Sales| 10000| 35|
| Evan|      Sales| 32000| 38|
| Fred|Engineering| 21000| 28|
| Alex|      Sales| 30000| 33|
|  Tom|Engineering| 23000| 33|
| Jane|  Marketing| 29000| 28|
| Jeff|  Marketing| 35000| 38|
| Paul|Engineering| 29000| 23|
|Chloe|Engineering| 23000| 25|
+-----+-----------+------+---+



In [11]:
## 부서별 급여의 총 합을 구하시오 -> groupBy
sqlCtx.sql( 'select dept, sum(salary) from employees group by dept').show()

+-----------+-----------+
|       dept|sum(salary)|
+-----------+-----------+
|      Sales|      72000|
|Engineering|      96000|
|  Marketing|      64000|
+-----------+-----------+



# Partition
- GroupBy와 다르다

In [ ]:
## 부서별로 급여가 많은 순위를 구하시오 -> ? " partition! "

In [12]:
# sql 표준 구문
sql='''
     SELECT name, dept, 
     RANK() OVER (PARTITION BY dept ORDER BY salary) AS rank 
     FROM employees
     '''
sqlCtx.sql( sql ).show()

+-----+-----------+----+
| name|       dept|rank|
+-----+-----------+----+
| Lisa|      Sales|   1|
| Alex|      Sales|   2|
| Evan|      Sales|   3|
| Fred|Engineering|   1|
|  Tom|Engineering|   2|
|Chloe|Engineering|   2|
| Paul|Engineering|   4|
| Jane|  Marketing|   1|
| Jeff|  Marketing|   2|
+-----+-----------+----+



In [15]:
sqlCtx.sql( 'select max(salary) from employees').show()

+-----------+
|max(salary)|
+-----------+
|      35000|
+-----------+



In [16]:
empDF.selectExpr( 'max(salary)').show()

+-----------+
|max(salary)|
+-----------+
|      35000|
+-----------+



In [20]:
empDF.selectExpr('name', 
                 'dept', 
                 'RANK() OVER (PARTITION BY dept ORDER BY salary) AS rank ').show()

+-----+-----------+----+
| name|       dept|rank|
+-----+-----------+----+
| Lisa|      Sales|   1|
| Alex|      Sales|   2|
| Evan|      Sales|   3|
| Fred|Engineering|   1|
|  Tom|Engineering|   2|
|Chloe|Engineering|   2|
| Paul|Engineering|   4|
| Jane|  Marketing|   1|
| Jeff|  Marketing|   2|
+-----+-----------+----+



In [28]:
# sql 표준 구문
sql='''
     SELECT name, dept, 
     max(salary) OVER (PARTITION BY dept ORDER BY salary) AS salary_max 
     FROM employees
     '''
sqlCtx.sql( sql ).show()

+-----+-----------+----------+
| name|       dept|salary_max|
+-----+-----------+----------+
| Lisa|      Sales|     10000|
| Alex|      Sales|     30000|
| Evan|      Sales|     32000|
| Fred|Engineering|     21000|
|  Tom|Engineering|     23000|
|Chloe|Engineering|     23000|
| Paul|Engineering|     29000|
| Jane|  Marketing|     29000|
| Jeff|  Marketing|     35000|
+-----+-----------+----------+

